# Scala Smart Generation: Автоматический расчет размера файлов

**Исправления (v5):**
1.  **Calibration Fix**: Временные файлы пишутся в `spark.sql.warehouse.dir`, чтобы гарантировать использование той же файловой системы (HDFS/Local), что и основная таблица.
2.  **Verification Fix**: Путь к таблице берется из метаданных каталога (`spark.sessionState.catalog`), а не конструируется вручную.

In [1]:
import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.hadoop.fs.{FileSystem, Path}

val spark = SparkSession.builder
    .appName("ScalaSmartScale_v2")
    .master("spark://spark-master:7077")
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083")
    .enableHiveSupport()
    .getOrCreate()

println("Spark Session Created")
println("Default FS: " + spark.sparkContext.hadoopConfiguration.get("fs.defaultFS"))

Spark Session Created
Default FS: file:///


spark = org.apache.spark.sql.SparkSession@278de205


org.apache.spark.sql.SparkSession@278de205

In [2]:
/**
  * Функция калибровки. Считает оптимальное кол-во строк на файл.
  */
def calculateMaxRecordsPerFile(df: DataFrame, targetSizeMB: Int = 128, sampleFraction: Double = 0.01): Int = {
    println("--- Запуск калибровки ---")

    val sampleDf = df.sample(withReplacement = false, fraction = sampleFraction, seed = 42).cache()
    val sampleCount = sampleDf.count()
    
    if (sampleCount == 0) return 1000000

    // ИСПОЛЬЗУЕМ WAREHOUSE DIR ДЛЯ ВРЕМЕННЫХ ФАЙЛОВ
    // Это гарантирует, что мы пишем и читаем из одной и той же файловой системы.
    val warehouseDir = spark.conf.get("spark.sql.warehouse.dir", "/user/hive/warehouse")
    val tempPathString = s"$warehouseDir/temp_calib_${java.util.UUID.randomUUID.toString}"
    val tempPath = new Path(tempPathString)
    
    println(s"Temp calibration path: $tempPath")

    try {
        sampleDf.write.mode("overwrite").parquet(tempPathString)
        
        // ПОЛУЧАЕМ FS ИЗ ПУТИ
        val fs = tempPath.getFileSystem(spark.sparkContext.hadoopConfiguration)
        println(s"Calibration FileSystem: ${fs.getScheme}://${fs.getUri.getAuthority}")

        val contentSummary = fs.getContentSummary(tempPath)
        val totalBytes = contentSummary.getLength
        
        println(s"Сэмпл: $sampleCount строк")
        println(s"Размер на диске: ${totalBytes} байт (${totalBytes/1024/1024.0} MB)")

        // Чистим за собой
        fs.delete(tempPath, true)
        sampleDf.unpersist()

        if (totalBytes == 0) return 1000000 // Защита от 0

        val bytesPerRow = totalBytes.toDouble / sampleCount
        val targetBytes = targetSizeMB * 1024 * 1024L
        val estimatedRecords = (targetBytes / bytesPerRow).toInt
        
        println(s"Средний размер строки: ${f"$bytesPerRow%.2f"} байт")
        println(s"Рекомендуемый maxRecords: $estimatedRecords")
        println("-------------------------")
        
        estimatedRecords
    } catch {
        case e: Exception =>
            println("Ошибка калибровки: " + e.getMessage)
            e.printStackTrace()
            1000000
    }
}

calculateMaxRecordsPerFile: (df: org.apache.spark.sql.DataFrame, targetSizeMB: Int, sampleFraction: Double)Int


In [3]:
val numRows = 5000000L 
val df = spark.range(numRows).withColumnRenamed("id", "row_id")
    .withColumn("heavy_text_1", concat(lit("desc_"), hex(expr("rand()*1000000").cast("long"))))
    .withColumn("heavy_text_2", when(rand() > 0.5, lit("A long filler text string to simulate volume")).otherwise(lit(null)))
    .withColumn("numbers", (rand() * 10000).cast("double"))

println(s"Датасет определен: $numRows строк")

Датасет определен: 5000000 строк


numRows = 5000000
df = [row_id: bigint, heavy_text_1: string ... 2 more fields]


[row_id: bigint, heavy_text_1: string ... 2 more fields]

In [4]:
// 1. Калибровка
val optimalMaxRecords = calculateMaxRecordsPerFile(df, targetSizeMB = 128, sampleFraction = 0.05)

val tableName = "scala_smart_table_v2"

// 2. Запись
println(s"Записываем таблицу $tableName...")
df.write
  .mode("overwrite")
  .option("maxRecordsPerFile", optimalMaxRecords)
  .saveAsTable(tableName)
println("Запись завершена.")

--- Запуск калибровки ---
Temp calibration path: file:/home/jovyan/work/spark-warehouse/temp_calib_4cd5cae6-9deb-4978-b84d-8b099bb2f3c8
Ошибка калибровки: Job aborted due to stage failure: Task 0 in stage 4.0 failed 4 times, most recent failure: Lost task 0.3 in stage 4.0 (TID 11) (172.18.0.8 executor 0): java.io.IOException: Mkdirs failed to create file:/home/jovyan/work/spark-warehouse/temp_calib_4cd5cae6-9deb-4978-b84d-8b099bb2f3c8/_temporary/0/_temporary/attempt_202601182301282926008902230024666_0004_m_000000_11 (exists=false, cwd=file:/opt/spark/work/app-20260118230120-0009/0)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.parquet.hadoop.util.HadoopOutputFile.create(HadoopOutputFile.java:74)
	at org.apache.parquet.hadoop.Parque

optimalMaxRecords = 1000000
tableName = scala_smart_table_v2


scala_smart_table_v2

In [5]:
// 3. Проверка результата
import org.apache.spark.sql.catalyst.TableIdentifier

// Правильный способ узнать путь к таблице из метастора
val tableMetadata = spark.sessionState.catalog.getTableMetadata(TableIdentifier(tableName))
val tableUri = tableMetadata.location
val tablePath = new Path(tableUri)

// Получаем правильную FS для этого пути
val fs = tablePath.getFileSystem(spark.sparkContext.hadoopConfiguration)

println(s"Table Location: $tableUri")

println("Файлы:")
fs.listStatus(tablePath).foreach { status =>
    if (status.isFile && status.getPath.getName.startsWith("part")) {
        val sizeMB = status.getLen / 1024.0 / 1024.0
        println(s"File: ${status.getPath.getName}, Size: ${f"$sizeMB%.2f"} MB")
    }
}

Table Location: hdfs://namenode:9000/user/hive/warehouse/scala_smart_table_v2
Файлы:
File: part-00000-6684cc02-05eb-49e2-a943-eb7577246df4-c000.snappy.parquet, Size: 17.69 MB
File: part-00000-6684cc02-05eb-49e2-a943-eb7577246df4-c001.snappy.parquet, Size: 17.69 MB
File: part-00000-6684cc02-05eb-49e2-a943-eb7577246df4-c002.snappy.parquet, Size: 8.85 MB
File: part-00001-6684cc02-05eb-49e2-a943-eb7577246df4-c000.snappy.parquet, Size: 17.69 MB
File: part-00001-6684cc02-05eb-49e2-a943-eb7577246df4-c001.snappy.parquet, Size: 17.69 MB
File: part-00001-6684cc02-05eb-49e2-a943-eb7577246df4-c002.snappy.parquet, Size: 8.85 MB


tableMetadata = 


CatalogTable(
Catalog: spark_catalog
Database: default
Table: scala_smart_table_v2
Owner: root
Created Time: Sun Jan 18 23:01:32 UTC 2026
Last Access: UNKNOWN
Created By: Spark 3.5.1
Type: MANAGED
Provider: parquet
Statistics: 92768759 bytes
Location: hdfs://namenode:9000/user/hive/warehouse/scala_smart_table_v2
Serde Library: org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe
InputFormat: org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat
OutputFormat: org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat
Storage Properties: [maxRecordsPerFile=1000000]
Schema: root
 |-- row_id: long (nullable = true)
 |-- heavy_text_1: string (nullable = ...
